In [1]:
import requests
import json

class ResearchAgent:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "https://serpapi.com/search"  # SerpApi base URL
    
    def search(self, query, num_results=10):
        """
        Perform a search query using SerpApi.
        """
        params = {
            "q": query,
            "api_key": self.api_key,
            "num": num_results,
        }
        try:
            response = requests.get(self.base_url, params=params)
            response.raise_for_status()
            return response.json()  # Return the search results
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data: {e}")
            return None
    
    def gather_data(self, company_name, industry):
        """
        Gather data about the company and its industry.
        """
        queries = {
            "industry_trends": f"Emerging trends, AI adoption, and regulatory standards in the {industry} industry (2024)",
            "company_focus": f"{company_name} strategic focus areas and goals",
            "competitor_analysis": f"Top competitors of {company_name} in {industry} industry",
            "technology_integration": f"Applications of GenAI, LLMs, and automation in {industry} industry",
            "key_offerings": f"Key offerings and services by {company_name}",
            "vision_and_product_info": f"Vision, mission, and product details for {company_name} in the {industry} industry",
        }
        
        results = {}
        for key, query in queries.items():
            print(f"Searching: {query}")
            results[key] = self.search(query)
        
        return results

    def generate_summary(self, data):
        """
        Generate a summary from the gathered data.
        """
        summary = []
        for key, result in data.items():
            if not result:
                summary.append(f"No results found for {key.replace('_', ' ').title()}.")
                continue
            summary.append(f"### {key.replace('_', ' ').title()} ###")
            for i, item in enumerate(result.get("organic_results", []), start=1):
                title = item.get("title", "No Title")
                link = item.get("link", "No Link")
                snippet = item.get("snippet", "No Snippet")
                summary.append(f"{i}. **{title}**\n   {snippet}\n   [Read more]({link})\n")
        
        return "\n".join(summary)

    def propose_use_cases(self, data, company_name, industry):
        """
        Propose relevant use cases based on the analyzed data.
        """
        use_cases = [f"# Suggested Use Cases for {company_name} in the {industry} Industry\n"]
        
        trends = data.get("industry_trends", {}).get("organic_results", [])
        tech_integration = data.get("technology_integration", {}).get("organic_results", [])
        key_offerings = data.get("key_offerings", {}).get("organic_results", [])
        
        if trends:
            use_cases.append("## Industry Trends and Standards Insights\n")
            for item in trends[:3]:  # Include top 3 trends
                use_cases.append(f"- {item.get('title', 'Trend')}: {item.get('snippet', 'No description available')}")

        if key_offerings:
            use_cases.append("\n## Key Offerings\n")
            for item in key_offerings[:3]:  # Include top 3 key offerings
                use_cases.append(f"- {item.get('title', 'Offering')}: {item.get('snippet', 'No description available')}")

        use_cases.append("\n## Proposed Use Cases\n")
        if tech_integration:
            use_cases.append("### Leveraging AI, ML, and Automation\n")
            for item in tech_integration[:3]:  # Include top 3 use cases
                use_cases.append(f"- {item.get('title', 'Use Case')}: {item.get('snippet', 'No description available')}")
        else:
            use_cases.append("- Automate repetitive tasks using RPA and AI-driven workflows.\n")
            use_cases.append("- Enhance customer satisfaction through personalized chatbots powered by GenAI.\n")
            use_cases.append("- Implement predictive analytics for maintenance and inventory optimization.\n")
        
        return "\n".join(use_cases)

# Main program
if __name__ == "__main__":
    # Replace 'your_serpapi_key_here' with your SerpApi key
    api_key = "5402a7bfd62fcf4fa0b44ea525ce9b7093938358b0c62f86757ee96f82b6815e"
    agent = ResearchAgent(api_key)
    
    # Ask user for company and industry details
    company_name = input("Enter the company name: ").strip()
    industry = input("Enter the industry: ").strip()
    
    # Gather data
    research_data = agent.gather_data(company_name, industry)
    
    # Generate a summary report
    summary_report = agent.generate_summary(research_data)
    output_file = f"{company_name}_research_summary.md"
    with open(output_file, "w", encoding="utf-8") as file:
        file.write(summary_report)
    print(f"Research summary saved to {output_file}")
    
    # Propose relevant use cases
    use_case_report = agent.propose_use_cases(research_data, company_name, industry)
    use_case_file = f"{company_name}_use_cases.md"
    with open(use_case_file, "w", encoding="utf-8") as file:
        file.write(use_case_report)
    print(f"Use cases saved to {use_case_file}")


Searching: Emerging trends, AI adoption, and regulatory standards in the Automotive industry (2024)
Searching: Ford strategic focus areas and goals
Searching: Top competitors of Ford in Automotive industry
Searching: Applications of GenAI, LLMs, and automation in Automotive industry
Searching: Key offerings and services by Ford
Searching: Vision, mission, and product details for Ford in the Automotive industry
Research summary saved to Ford_research_summary.md
Use cases saved to Ford_use_cases.md


In [2]:
import os
from dotenv import load_dotenv
from groq import Groq

class GenAIQuerySystem:
    def __init__(self, folder_path):
        """
        Initialize the GenAI Query System by loading all text files from the specified folder.
        Then, initialize the Groq API client.
        """
        load_dotenv()  # Load environment variables from the .env file
        groq_api_key = os.getenv("GROQ_API_KEY")
        if not groq_api_key:
            raise ValueError("Groq API Key not found. Please set GROQ_API_KEY in your .env file.")
        
        self.groq_client = Groq(api_key=groq_api_key)
        self.context = self.load_context_from_folder(folder_path)
        
        # Fine-tuning model using your dataset
        self.model = self.fine_tune_model(self.context)

    @staticmethod
    def load_context_from_folder(folder_path):
        """
        Load and combine all text content from files in the specified folder.
        """
        combined_context = []
        for root, _, files in os.walk(folder_path):
            for file in files:
                if file.endswith(".txt"):
                    file_path = os.path.join(root, file)
                    with open(file_path, "r", encoding="utf-8") as f:
                        combined_context.append(f.read())
        return "\n".join(combined_context)

    def fine_tune_model(self, context):
        """
        Fine-tune a Groq model using the provided context data.
        """
        try:
            fine_tuned_model = self.groq_client.train_model(data=context, task="text-generation")
            return fine_tuned_model
        except Exception as e:
            print(f"An error occurred while fine-tuning the model: {e}")
            return None
    
    def answer_question(self, question):
        """
        Answer a question using the fine-tuned Groq model and the loaded context.
        """
        if self.model:
            try:
                response = self.groq_client.query_model(model=self.model, query=question, context=self.context)
                return response['answer']
            except Exception as e:
                print(f"An error occurred while answering the question: {e}")
                return None
        else:
            print("Model not fine-tuned. Please check the training process.")
            return None

if __name__ == "__main__":
    # Path to the folder containing training data
    folder_path = "results"
    try:
        gen_ai_system = GenAIQuerySystem(folder_path)

        print("Ask me questions about leveraging GenAI, LLMs, and ML technologies.")
        while True:
            question = input("\nYour question (type 'exit' to quit): ").strip()
            if question.lower() == "exit":
                print("Goodbye!")
                break
            try:
                answer = gen_ai_system.answer_question(question)
                print(f"Answer: {answer}")
            except Exception as e:
                print(f"An error occurred: {e}")
    except ValueError as e:
        print(e)


An error occurred while fine-tuning the model: 'Groq' object has no attribute 'train_model'
Ask me questions about leveraging GenAI, LLMs, and ML technologies.
Model not fine-tuned. Please check the training process.
Answer: None
Model not fine-tuned. Please check the training process.
Answer: None
Model not fine-tuned. Please check the training process.
Answer: None
Goodbye!


In [8]:
class UseCaseGenerator:
    def __init__(self):
        pass

    def analyze_data(self, research_data):
        """
        Analyze research data to identify patterns and opportunities.
        """
        use_cases = []
        if not research_data:
            print("No research data provided.")
            return []

        # Example use case generation based on keywords in research data
        for key, result in research_data.items():
            if not result:
                continue

            print(f"Analyzing {key}...")
            for item in result.get("organic_results", []):
                snippet = item.get("snippet", "").lower()

                # Identify potential use cases based on common keywords
                if "automation" in snippet:
                    use_cases.append("Implement automation for repetitive tasks.")
                if "predictive" in snippet:
                    use_cases.append("Develop predictive models for key business metrics.")
                if "customer" in snippet:
                    use_cases.append("Build a customer service chatbot to enhance engagement.")
                if "supply chain" in snippet:
                    use_cases.append("Use AI for supply chain optimization.")

        return list(set(use_cases))  # Return unique use cases

    def find_resources(self, use_cases):
        """
        Search for relevant datasets and tools for each use case.
        """
        resources = {}
        for use_case in use_cases:
            print(f"Finding resources for: {use_case}")

            # Example resource mapping
            if "automation" in use_case:
                resources[use_case] = {
                    "datasets": ["https://www.kaggle.com/datasets"],
                    "tools": ["https://www.uipath.com/", "https://airflow.apache.org/"],
                }
            elif "predictive" in use_case:
                resources[use_case] = {
                    "datasets": ["https://www.kaggle.com/datasets/gpred/predictive-maintenance"],
                    "tools": ["https://www.tensorflow.org/", "https://scikit-learn.org/"],
                }
            elif "customer service" in use_case:
                resources[use_case] = {
                    "datasets": ["https://www.huggingface.co/datasets/daily_dialog"],
                    "tools": ["https://dialogflow.cloud.google.com/", "https://rasa.com/"],
                }
            elif "supply chain" in use_case:
                resources[use_case] = {
                    "datasets": ["https://data.world/datasets/supply-chain"],
                    "tools": ["https://www.anylogistix.com/", "https://or-tools.github.io/"],
                }

        return resources

    def generate_report(self, use_cases, resources):
        """
        Create a detailed report with use cases and their resources.
        """
        report = ["# AI/ML Use Cases and Resources\n"]
        for use_case in use_cases:
            report.append(f"## Use Case: {use_case}\n")
            resource = resources.get(use_case, {})
            datasets = resource.get("datasets", [])
            tools = resource.get("tools", [])

            report.append("### Datasets:\n")
            report.extend([f"- [{url}]({url})\n" for url in datasets])

            report.append("\n### Tools:\n")
            report.extend([f"- [{url}]({url})\n" for url in tools])

            report.append("\n---\n")

        return "\n".join(report)


# Usage Example
if __name__ == "__main__":
    # Assume `research_data` is the output from the previous ResearchAgent step
    with open("research_summary.md", "r", encoding="utf-8") as file:
        research_summary = file.read()
    
    # Convert the research summary into dummy structured data for demo purposes
    # (Replace this with the actual JSON output from the `ResearchAgent`)
    research_data = {
        "industry_trends": {
            "organic_results": [{"snippet": "AI automation is revolutionizing the supply chain in 2024."}],
        },
        "company_focus": {
            "organic_results": [{"snippet": "Tesla focuses on predictive maintenance for vehicles."}],
        },
        "competitor_analysis": {
            "organic_results": [{"snippet": "Competitors are using AI chatbots for customer service."}],
        },
    }

    generator = UseCaseGenerator()

    # Generate use cases
    use_cases = generator.analyze_data(research_data)

    # Find resources for the use cases
    resources = generator.find_resources(use_cases)

    # Generate a report
    report = generator.generate_report(use_cases, resources)

    # Save the report to a markdown file
    with open("use_case_report.md", "w", encoding="utf-8") as file:
        file.write(report)

    print("Use case report saved to use_case_report.md")


Analyzing industry_trends...
Analyzing company_focus...
Analyzing competitor_analysis...
Finding resources for: Implement automation for repetitive tasks.
Finding resources for: Develop predictive models for key business metrics.
Finding resources for: Use AI for supply chain optimization.
Finding resources for: Build a customer service chatbot to enhance engagement.
Use case report saved to use_case_report.md


In [9]:
import datetime

class ActionPlanGenerator:
    def __init__(self):
        pass

    def prioritize_use_cases(self, use_cases):
        """
        Prioritize use cases based on potential impact and feasibility.
        """
        priorities = {}
        for use_case in use_cases:
            print(f"Evaluating: {use_case}")

            # Dummy prioritization logic (can be replaced with real criteria)
            if "automation" in use_case:
                priorities[use_case] = {"priority": "High", "estimated_timeline": "3 months"}
            elif "predictive" in use_case:
                priorities[use_case] = {"priority": "Medium", "estimated_timeline": "4 months"}
            elif "customer service" in use_case:
                priorities[use_case] = {"priority": "High", "estimated_timeline": "2 months"}
            elif "supply chain" in use_case:
                priorities[use_case] = {"priority": "Medium", "estimated_timeline": "5 months"}
            else:
                priorities[use_case] = {"priority": "Low", "estimated_timeline": "6+ months"}

        return priorities

    def generate_action_plan(self, use_cases, resources, priorities):
        """
        Create an action plan for implementing the use cases.
        """
        action_plan = ["# Action Plan for AI/ML Implementation\n"]
        today = datetime.date.today()

        for use_case in use_cases:
            priority_info = priorities.get(use_case, {})
            resources_info = resources.get(use_case, {})
            datasets = resources_info.get("datasets", [])
            tools = resources_info.get("tools", [])

            action_plan.append(f"## Use Case: {use_case}\n")
            action_plan.append(f"**Priority:** {priority_info.get('priority', 'Unknown')}\n")
            action_plan.append(f"**Estimated Timeline:** {priority_info.get('estimated_timeline', 'Unknown')}\n")
            action_plan.append(f"**Start Date:** {today.strftime('%Y-%m-%d')}\n")

            action_plan.append("\n### Resources:\n")
            action_plan.append("#### Datasets:\n")
            action_plan.extend([f"- [{url}]({url})\n" for url in datasets])

            action_plan.append("\n#### Tools:\n")
            action_plan.extend([f"- [{url}]({url})\n" for url in tools])

            action_plan.append("\n---\n")

        return "\n".join(action_plan)


# Usage Example
if __name__ == "__main__":
    # Use cases and resources from the previous step
    use_cases = [
        "Implement automation for repetitive tasks.",
        "Develop predictive models for key business metrics.",
        "Build a customer service chatbot to enhance engagement.",
        "Use AI for supply chain optimization.",
    ]

    resources = {
        "Implement automation for repetitive tasks.": {
            "datasets": ["https://www.kaggle.com/datasets"],
            "tools": ["https://www.uipath.com/", "https://airflow.apache.org/"],
        },
        "Develop predictive models for key business metrics.": {
            "datasets": ["https://www.kaggle.com/datasets/gpred/predictive-maintenance"],
            "tools": ["https://www.tensorflow.org/", "https://scikit-learn.org/"],
        },
        "Build a customer service chatbot to enhance engagement.": {
            "datasets": ["https://www.huggingface.co/datasets/daily_dialog"],
            "tools": ["https://dialogflow.cloud.google.com/", "https://rasa.com/"],
        },
        "Use AI for supply chain optimization.": {
            "datasets": ["https://data.world/datasets/supply-chain"],
            "tools": ["https://www.anylogistix.com/", "https://or-tools.github.io/"],
        },
    }

    # Initialize the action plan generator
    generator = ActionPlanGenerator()

    # Prioritize use cases
    priorities = generator.prioritize_use_cases(use_cases)

    # Generate an action plan
    action_plan = generator.generate_action_plan(use_cases, resources, priorities)

    # Save the action plan to a markdown file
    with open("action_plan.md", "w", encoding="utf-8") as file:
        file.write(action_plan)

    print("Action plan saved to action_plan.md")


Evaluating: Implement automation for repetitive tasks.
Evaluating: Develop predictive models for key business metrics.
Evaluating: Build a customer service chatbot to enhance engagement.
Evaluating: Use AI for supply chain optimization.
Action plan saved to action_plan.md


In [10]:
class ReviewReportGenerator:
    def __init__(self, action_plan_file):
        self.action_plan_file = action_plan_file

    def extract_highlights(self):
        """
        Extract key highlights from the action plan.
        """
        highlights = []
        with open(self.action_plan_file, "r", encoding="utf-8") as file:
            lines = file.readlines()
            for line in lines:
                if line.startswith("## Use Case:"):
                    highlights.append(line.strip())
                elif "Priority" in line or "Estimated Timeline" in line:
                    highlights.append(line.strip())
        return highlights

    def generate_review_report(self, highlights):
        """
        Create a review report for stakeholders.
        """
        report = ["# Action Plan Review Report\n"]
        report.append("## Highlights\n")
        report.extend([f"- {highlight}\n" for highlight in highlights])

        report.append("\n## Key Questions\n")
        report.append("- Are the proposed use cases aligned with company goals?\n")
        report.append("- Are the timelines realistic?\n")
        report.append("- Are the resources appropriate for implementation?\n")

        report.append("\n## Areas Requiring Feedback\n")
        report.append("- Suggestions for prioritizing use cases.\n")
        report.append("- Additional resources or tools to consider.\n")

        return "\n".join(report)

# Usage Example
if __name__ == "__main__":
    # Path to the previously generated action plan file
    action_plan_file = "action_plan.md"

    # Initialize the review report generator
    review_generator = ReviewReportGenerator(action_plan_file)

    # Extract highlights
    highlights = review_generator.extract_highlights()

    # Generate the review report
    review_report = review_generator.generate_review_report(highlights)

    # Save the review report to a markdown file
    with open("review_report.md", "w", encoding="utf-8") as file:
        file.write(review_report)

    print("Review report saved to review_report.md")



Review report saved to review_report.md
